# Validaciones Diarias Sistemas de Información UT

## CONFIGURACIONES Y CARGA DE ARCHIVOS


### IMPORTACIÓN DE LIBRERIAS


In [ ]:
import pandas as pd
from datetime import datetime
import pytz
import smtplib
from email.message import EmailMessage

### CARGA DE ARCHIVOS

Este modulo permite la carga de los archivos en formato xlsx y csv según el caso e informa cuantas filas y columnas tiene el archivo cargado.

Archivos Necesarios


*   PQRS_Formato_Servicios (Umbrella)
*   PR_Formato (Umbrella) mes actual
*   GEN_Sitios_Activos_SI (Umbrella)
*   Lista de categorias (Umbrella)
*   LOG PQRSD (NetAdvisor) desde 01-01-2023 a fecha actual

In [ ]:
# Cargue PQRSD_Formato_Servicios
path ='/content/PQRSD_Formato_Servicios-2024-12-16.xlsx'
data = pd.read_excel(path)
data.shape

#Usar en caso de ejecutar localmente
#path2 = 'C:/Users/William Wallace/Desktop/PQRSD_Formato_Servicios-2024-07-24.xlsx'
#data_Categorias = pd.read_excel(path2)

In [ ]:
# Cargue paradas de reloj del mes - PR_Formato

path3 = '/content/PR_Formato-2024-12-16.xlsx'
data_paradas = pd.read_excel(path3)
data_paradas.shape

#Usar en caso de ejecutar localmente
#path3 = 'C:/Users/William Wallace/Desktop/PR_Formato-2024-07-24.xlsx'

In [ ]:
# Cargue lista personalizada - GEN_Sitios_Activos_SI
path4 = '/content/GEN_Sitios_Activos_SI-2024-12-14.xlsx'
data_id_punto = pd.read_excel(path4)
data_id_punto.shape

#Usar en caso de ejecutar localmente
#path4 = 'C:/Users/William Wallace/Desktop/GEN_Sitios_Activos_SI-2024-07-24.xlsx'

In [ ]:
# Cargue LOG PQRSD (NetAdvisor)
path5 = '/content/NetAdvisor 2024-12-16T14_07_16.398Z.csv'
log_pqrs_datawifi = pd.read_csv(path5)
log_pqrs_datawifi.shape

#Usar en caso de ejecutar localmente
#path5 = 'C:/Users/William Wallace/Desktop/GEN_Sitios_Activos_SI-2024-07-24.xlsx'

In [ ]:
# Cargue lista categoria y subcategoria
path ='/content/Listas Personalizadas Listado Categorias PQRS_2024-12-14_11-33-58.xlsx'
categorias_subcategorias = pd.read_excel(path)
categorias_subcategorias.shape

### CONFIGURACIONES DATAFRAMES

In [ ]:
# Configurar pandas para mostrar todas las columnas
pd.set_option('display.max_rows', None)

#dataframe reporte PQRSD_Formato_Servicios
dataframe = pd.DataFrame(data,columns=data.columns)
dataframe['Fecha_Cierre_PQRS'] = pd.to_datetime(dataframe['Fecha_Cierre_PQRS'])
dataframe['Hora_Cierre'] = dataframe['Fecha_Cierre_PQRS'].dt.hour.fillna(-1).astype(int)
dataframe['dia_fecha_Cierre'] = dataframe['Fecha_Cierre_PQRS'].dt.day.fillna(-1).astype(int)
dataframe['mes_fecha_Cierre'] = dataframe['Fecha_Cierre_PQRS'].dt.month.fillna(-1).astype(int)
dataframe['año_fecha_Cierre'] = dataframe['Fecha_Cierre_PQRS'].dt.year.fillna(-1).astype(int)

dataframe['Fecha_Creacion'] = pd.to_datetime(dataframe['Fecha_Creacion'])
dataframe['Hora_Creacion'] = dataframe['Fecha_Creacion'].dt.hour.fillna(-1).astype(int)
dataframe['dia_Fecha_Creacion'] = dataframe['Fecha_Creacion'].dt.day.fillna(-1).astype(int)
dataframe['mes_Fecha_Creacion'] = dataframe['Fecha_Creacion'].dt.month.fillna(-1).astype(int)
dataframe['año_Fecha_Creacion'] = dataframe['Fecha_Creacion'].dt.year.fillna(-1).astype(int)

dataframe['fecha_creacion_agenda'] = pd.to_datetime(dataframe['fecha_creacion_agenda'])

# Función para extraer horas de una cadena de tiempo
def extraer_horas(tiempo):
    partes = tiempo.split(':')
    horas = int(partes[0])
    return horas

#dataframe reporte paradas de reloj del mes
dataframe_Paradas = pd.DataFrame(data_paradas,columns=data_paradas.columns)
dataframe_Paradas['Hora_total_parada'] = dataframe_Paradas['tiempo_parada'].apply(extraer_horas)

dataframe_Paradas['fecha_ceracion_pqrs'] = pd.to_datetime(dataframe_Paradas['fecha_ceracion_pqrs'])
dataframe_Paradas['fecha_cierre_ticket'] = pd.to_datetime(dataframe_Paradas['fecha_cierre_ticket'])

dataframe_Paradas['dateparada'] = pd.to_datetime(dataframe_Paradas['dateparada'])
dataframe_Paradas['Dia_Parada'] = dataframe_Paradas['dateparada'].dt.day
dataframe_Paradas['dia_semana'] = dataframe_Paradas['dateparada'].dt.dayofweek
dataframe_Paradas['Hora_inicio'] = dataframe_Paradas['dateparada'].dt.hour
dataframe_Paradas['finishparada'] = pd.to_datetime(dataframe_Paradas['finishparada'])
dataframe_Paradas['Hora_fin'] = dataframe_Paradas['finishparada'].dt.hour

# Configuraciones de fechas
fecha_actual = datetime.now()
numero_dia = fecha_actual.day
mes_actual = fecha_actual.month
año_actual = fecha_actual.year
fecha = datetime.now().date()

dataframe_horarios = pd.DataFrame(data_id_punto,columns=data_id_punto.columns)

dataframe_log_pqrs_datawifi = pd.DataFrame(log_pqrs_datawifi,columns=log_pqrs_datawifi.columns)

## VALIDACIONES


### Validación 1 - PQRSD SIN JOBID

Esta validación se encarga de verificar si existen mas de 119 PQRSD sin jobid.   
Retorna:
*    PQRSD sin jobid
*    Mensaje de error o exito.

In [ ]:
#validacion 1 PQRSD sin JOBID

data['Fecha_Creacion'] = pd.to_datetime(data['Fecha_Creacion'], errors='coerce')
nulos = data[(data['sv_job_id'].isna()) & (data['Fecha_Creacion'] > '2023-01-01')]
total = nulos.shape[0]

nulos = nulos[['ID','GRUPO','PRIORIDAD']]

if nulos.shape[0] > 0:
    nulos = nulos.to_html(index=False)
    #print ("Revisar, hay PQRSD agendados sin parada Fuera de Horario MDA")
    error1= f"Total: <b>{total}</b><br>{nulos}"
else:
    #print ("todos los pqrsd agendados tienen parada de reloj")
    error1="OK - Sin Novedades"

### Validación 2 - PQRSD SIN FECHA MAXIMA DE ATENCIÓN
Esta validación verifica que todos los PQRSD agendados tengan fecha maxima de atención.  
Retorna:
*    La cantidad de PQRSD agendados con fecha máxima
*    La cantidad de PQRSD agendados **SIN** fecha máxima
*    y mensaje de error o exito.

In [ ]:
df_filtrado = dataframe[(dataframe['Estado'] == "Agendar") & (dataframe['fecha_maxima_atencion'].notnull())]
total_agendados=df_filtrado.shape[0]
print ("Cantidad de PQRSD agendados con fecha maxima:")
print (total_agendados)
df_filtrado2 = dataframe[(dataframe['Estado'] == "Agendar") & (dataframe['fecha_maxima_atencion'].isnull())]
total_agendados2=df_filtrado2.shape[0]
print ("Cantidad de PQRSD agendados SIN fecha maxima:")
print (total_agendados2)

df_filtrado2 = df_filtrado2[['ID','GRUPO','PRIORIDAD']]

if df_filtrado2.shape[0] > 0:
    df_filtrado2 = df_filtrado2.to_html(index=False)
    error2= f"Total: <b>{total_agendados2}</b> <br>{df_filtrado2}"
else:
    error2="OK - Sin Novedades"

Cantidad de PQRSD agendados con fecha maxima:
2911
Cantidad de PQRSD agendados SIN fecha maxima:
0


### VALIDACIÓN 3 - PQRSD AGENDADOS SIN CORRECTIVO
Esta validación verifica que si el PQRSD esta en estado agendado tenga su respectivo Mantenimiento Correctivo.  
Retorna:
*    La cantidad de PQRSD agendados con correctivo.
*    La cantidad de PQRSD agendados **SIN** correctivo.
*    Mensaje de error o exito.

In [ ]:
#validacion3 PQRSD AGENDADO SIN CORRECTIVO

df_filtrado_validacion3 = dataframe[(dataframe['Estado'] == "Agendar") & (dataframe['CORRECTIVO'].notnull())]
#total_agendados=df_filtrado["Estado"].value_counts()
total_correctivos=df_filtrado_validacion3.shape[0]
print ("Cantidad de PQRSD agendados con correctivo:")
print (total_correctivos)
df_filtrado2_validacion3 = dataframe[(dataframe['Estado'] == "Agendar") & (dataframe['CORRECTIVO'].isnull())]
total_sin_correctivo=df_filtrado2_validacion3.shape[0]
print ("Cantidad de PQRSD agendados SIN correctivo:")
print (total_sin_correctivo)

df_filtrado2_validacion3 = df_filtrado2_validacion3[['ID','GRUPO','PRIORIDAD']]

if df_filtrado2_validacion3.shape[0] > 0:
    df_filtrado2_validacion3 = df_filtrado2_validacion3.to_html(index=False)
    error3= f"Total: <b>{total_sin_correctivo}</b> <br>{df_filtrado2_validacion3}"
else:
    error3="OK - Sin Novedades"

Cantidad de PQRSD agendados con correctivo:
2911
Cantidad de PQRSD agendados SIN correctivo:
0


### VALIDACIÓN 4 - CIERRES DE PQRSD FUERA DE HORARIO

Esta validación verifica si existen PQRSD de las subcategorias **CD-FALLA EN EJECUCIÓN DE PRUEBA DE VELOCIDAD 5 DÍAS CALENDARIO** o **CD-MEDICIÓN DIRECTA DE VELOCIDAD EFECTIVA DE TRANSMISIÓN DE DATOS** cerrados despues de las 21:00 y antes de las 06:00 del mes en curso.  

Retorna:
*    La cantidad de PQRSD cerrados fuera de horario.
*    Si existen PQRSD cerrados fuera de horario retorna el **UUID** y **GRUPO**.
*    Mensaje de error o exito.

In [ ]:
#validacion 4 Fecha de cierre - verificar cierres fuera de horario
subcategorias = ['CD-FALLA EN EJECUCIÓN DE PRUEBA DE VELOCIDAD 5 DÍAS CALENDARIO', 'CD-MEDICIÓN DIRECTA DE VELOCIDAD EFECTIVA DE TRANSMISIÓN DE DATOS']

df_filtrado_antes_validacion4 = dataframe[(dataframe['Estado'] == "Cerrar") & (dataframe['CATEGORIA'] == "CD-CENTROS DE GESTIÓN SERVICIO") & (dataframe['SUBCATEGORIA'].isin(subcategorias)) & (dataframe['Hora_Cierre'] >= 21) & ( dataframe['mes_fecha_Cierre'] == mes_actual ) &  ( dataframe['año_fecha_Cierre'] == año_actual ) ]
df_filtrado_despues_validacion4 = dataframe[(dataframe['Estado'] == "Cerrar") & (dataframe['CATEGORIA'] == "CD-CENTROS DE GESTIÓN SERVICIO") & (dataframe['SUBCATEGORIA'].isin(subcategorias)) & (dataframe['Hora_Cierre'] < 6) & ( dataframe['mes_fecha_Cierre'] == mes_actual ) & ( dataframe['año_fecha_Cierre'] == año_actual ) ]

Fuera_de_horario_antes=df_filtrado_antes_validacion4.shape[0]
Fuera_de_horario_despues=df_filtrado_despues_validacion4.shape[0]
Fuera_de_horario_total=Fuera_de_horario_antes+Fuera_de_horario_despues

print ("Cantidad de PQRSD cerrados antes de las 6:")
print(Fuera_de_horario_antes)

print ("Cantidad de PQRSD cerrados despues de las 21:")
Fuera_de_horario_despues

print ("Cantidad de PQRSD cerrados fuera de horario:")
print (Fuera_de_horario_total)

df_filtrado_antes_validacion4 = df_filtrado_antes_validacion4[['ID','GRUPO','PRIORIDAD','Fecha_Creacion','Fecha_Cierre_PQRS']]

if Fuera_de_horario_total > 0:
    df_filtrado_antes_validacion4 = df_filtrado_antes_validacion4.to_html(index=False)
    error4= f"Total: <b>{Fuera_de_horario_antes}</b> <br>{df_filtrado_antes_validacion4}"
else:
    error4="OK - Sin Novedades"

Cantidad de PQRSD cerrados antes de las 6:
4
Cantidad de PQRSD cerrados despues de las 21:
Cantidad de PQRSD cerrados fuera de horario:
4


### VALIDACIÓN 5 - PQRSD AGENDADOS SIN PARADA DE RELOJ FUERA DE HORARIO CD

Esta validación verifica que los PQRSD en estado **Agendar** tengan la parada de reloj Fuera de Horario CD.  

Retorna:
*    La cantidad de PQRSD sin parada de reloj Fuera de Horario CD.
*    Si existen PQRSD sin parada de reloj Fuera de Horario CD retorna el **UUID** y **GRUPO**.
*    Mensaje de error o exito.

In [ ]:
#validacion 5 verificar que todos los pqrsd agendados tienen parada de fuera de horario cd

df_filtrado_validacion5_agendados = dataframe[(dataframe['Estado'] == "Agendar")]
df_filtrado_validacion5_fuera_cd = dataframe_Paradas[(dataframe_Paradas['tipo_parada'] == "Fuera de Horario CD") & (dataframe_Paradas['Dia_Parada'] == numero_dia)]
resultvalidacion5 = pd.merge(df_filtrado_validacion5_agendados,df_filtrado_validacion5_fuera_cd, left_on='ID', right_on='uuid', how='left')
resultvalidacion5_2 = resultvalidacion5[(resultvalidacion5['uuid'].isnull())]
Sin_parada_cd=resultvalidacion5_2.shape[0]

resultvalidacion5_2 = resultvalidacion5_2[['ID','GRUPO','PRIORIDAD','Fecha_Creacion','Fecha_Cierre_PQRS']]

if Sin_parada_cd > 0:
    resultvalidacion5_2 = resultvalidacion5_2.to_html(index=False)
    error5= f"Total: <b>{Sin_parada_cd}</b> <br>{resultvalidacion5_2}"
else:
    error5="OK - Sin Novedades"

### VALIDACIÓN 6 - PQRSD ASIGNADOS A MESA DE AYUDA SIN PARADA DE RELOJ FUERA DE HORARIO MDA

Esta validación verifica que los PQRSD en estado **Asignado Mesa de Ayuda** tengan la parada de reloj Fuera de Horario MDA.  

Retorna:
*    La cantidad de PQRSD sin parada de reloj Fuera de Horario MDA.
*    Si existen PQRSD sin parada de reloj Fuera de Horario MDA retorna el **UUID** y **GRUPO**.
*    Mensaje de error o exito.

In [ ]:
#validacion 6 verificar que todos los pqrsd en mesa de ayuda tienen parada de fuera de horario MDA

df_filtrado_validacion6_agendados = dataframe[(dataframe['Estado'] == "Asignado Mesa de Ayuda")]
df_filtrado_validacion6_fuera_mda = dataframe_Paradas[(dataframe_Paradas['tipo_parada'] == "Fuera de Horario MDA") & (dataframe_Paradas['Dia_Parada'] == numero_dia)]
resultvalidacion6 = pd.merge(df_filtrado_validacion6_agendados,df_filtrado_validacion6_fuera_mda, left_on='ID', right_on='uuid', how='left')
resultvalidacion6_2 = resultvalidacion6[(resultvalidacion6['uuid'].isnull())]

Sin_parada_mda=resultvalidacion6_2.shape[0]

resultvalidacion6_2 = resultvalidacion6_2[['ID','GRUPO','PRIORIDAD','Fecha_Creacion','Fecha_Cierre_PQRS']]

if Sin_parada_mda > 0:
    resultvalidacion6_2 = resultvalidacion6_2.to_html(index=False)
    error6= f"Total: <b>{Sin_parada_mda}</b> <br>{resultvalidacion6_2}"
else:
    error6="OK - Sin Novedades"


### VALIDACIÓN 7 - PARADAS DE RELOJ MAYOR A 24 HORAS

Esta validación verifica que **NO** existan paradas de reloj Fuera de Horario MDA o Fuera de Horario CD mayores a 24 Horas.  

Retorna:
*    La cantidad de Paradas de reloj mayores a 23:59 Horas.
*    Mensaje de error o exito.



In [ ]:
#validacion 7 paradas con tiempo mayor a 24 horas
dataframe_Paradas['Hora_total_parada']

df_filtrado_validacion7 = dataframe_Paradas[(dataframe_Paradas['Hora_total_parada'] > 24) & (dataframe_Paradas['tipo_parada'] != "Evento excepcional") & (dataframe_Paradas['tipo_parada'] != "Diagnostico 8 Dias") ]
exede23horas=df_filtrado_validacion7.shape[0]

df_filtrado_validacion7 = df_filtrado_validacion7[['id_parada','idbeneficiario','uuid','fecha_ceracion_pqrs','dateparada','finishparada','fecha_cierre_ticket']]

if exede23horas > 0:
    df_filtrado_validacion7 = df_filtrado_validacion7.to_html(index=False)
    error7= f"Total: <b>{exede23horas}</b> {df_filtrado_validacion7}"
else:
    error7="OK - Sin Novedades"

### VALIDACIÓN 8 - PARADAS DE RELOJ FUERA DE HORARIO CD - CUANDO EL CD NO TIENE HORARIOS  
Esta validación verifica los horarios de los Centros Digitales y realiza la validación con las paradas de reloj Fuera de Horario CD  

Retorna:  
*    Cantidad de paradas Fuera de horario CD dia por dia mal puestas.  
*    Mensaje de error o exito.

In [ ]:
#validacion 8 parada fuera de horario cd -fuera del horario cargado
#dataframe_horarios
#dataframe_Paradas

dataframe_horarios['Consolidado_Lunes'] = dataframe_horarios['Consolidado_Lunes'].fillna('00:00:00 a 00:00:00').astype(str)
dataframe_horarios['Consolidado_Martes'] = dataframe_horarios['Consolidado_Martes'].fillna('00:00:00 a 00:00:00').astype(str)
dataframe_horarios['Consolidado_Miercoles'] = dataframe_horarios['Consolidado_Miercoles'].fillna('00:00:00 a 00:00:00').astype(str)
dataframe_horarios['Consolidado_Jueves'] = dataframe_horarios['Consolidado_Jueves'].fillna('00:00:00 a 00:00:00').astype(str)
dataframe_horarios['Consolidado_Viernes'] = dataframe_horarios['Consolidado_Viernes'].fillna('00:00:00 a 00:00:00').astype(str)
dataframe_horarios['Consolidado_Sabado'] = dataframe_horarios['Consolidado_Sabado'].fillna('00:00:00 a 00:00:00').astype(str)
dataframe_horarios['Consolidado_Domingo'] = dataframe_horarios['Consolidado_Domingo'].fillna('00:00:00 a 00:00:00').astype(str)

def extraer_horas(rango_tiempo):
    # Dividir el rango de tiempo en dos partes
    try:
        tiempo_inicio, tiempo_fin = rango_tiempo.split(' a ')

    except ValueError as e:
        print(f"Error al convertir horas: {e}")
        return None, None
    return tiempo_inicio, tiempo_fin

# Aplicar la función a la columna 'tiempo' y crear nuevas columnas 'hora_inicio' y 'hora_fin'
dataframe_horarios[['Lunes_hora_inicio', 'Lunes_hora_fin']] = dataframe_horarios['Consolidado_Lunes'].apply(lambda x: pd.Series(extraer_horas(x)))
dataframe_horarios[['Martes_hora_inicio', 'Martes_hora_fin']] = dataframe_horarios['Consolidado_Martes'].apply(lambda x: pd.Series(extraer_horas(x)))
dataframe_horarios[['Miercoles_hora_inicio', 'Miercoles_hora_fin']] = dataframe_horarios['Consolidado_Miercoles'].apply(lambda x: pd.Series(extraer_horas(x)))
dataframe_horarios[['Jueves_hora_inicio', 'Jueves_hora_fin']] = dataframe_horarios['Consolidado_Jueves'].apply(lambda x: pd.Series(extraer_horas(x)))
dataframe_horarios[['Viernes_hora_inicio', 'Viernes_hora_fin']] = dataframe_horarios['Consolidado_Viernes'].apply(lambda x: pd.Series(extraer_horas(x)))
dataframe_horarios[['Sabado_hora_inicio', 'Sabado_hora_fin']] = dataframe_horarios['Consolidado_Sabado'].apply(lambda x: pd.Series(extraer_horas(x)))
dataframe_horarios[['Domingo_hora_inicio', 'Domingo_hora_fin']] = dataframe_horarios['Consolidado_Domingo'].apply(lambda x: pd.Series(extraer_horas(x)))

df_filtrado8_paradas_fuera_cd = dataframe_Paradas[(dataframe_Paradas['tipo_parada'] == "Fuera de Horario CD")]

resultvalidacion81 = pd.merge(df_filtrado8_paradas_fuera_cd,dataframe_horarios, left_on='idbeneficiario', right_on='Id_Beneficiario', how='left')
resultvalidacion8 = pd.merge(resultvalidacion81,dataframe, left_on='uuid', right_on='ID', how='inner')

resultvalidacion8['Hora_inicio'] = pd.to_numeric(resultvalidacion8['Hora_inicio'], errors='coerce')

resultvalidacion8['Lunes_hora_fin'] = pd.to_numeric(resultvalidacion8['Lunes_hora_fin'], errors='coerce')
resultvalidacion8['Lunes_hora_inicio'] = pd.to_numeric(resultvalidacion8['Lunes_hora_inicio'], errors='coerce')
resultvalidacion8['Martes_hora_fin'] = pd.to_numeric(resultvalidacion8['Martes_hora_fin'], errors='coerce')
resultvalidacion8['Martes_hora_inicio'] = pd.to_numeric(resultvalidacion8['Martes_hora_inicio'], errors='coerce')
resultvalidacion8['Miercoles_hora_fin'] = pd.to_numeric(resultvalidacion8['Miercoles_hora_fin'], errors='coerce')
resultvalidacion8['Miercoles_hora_inicio'] = pd.to_numeric(resultvalidacion8['Miercoles_hora_inicio'], errors='coerce')
resultvalidacion8['Jueves_hora_fin'] = pd.to_numeric(resultvalidacion8['Jueves_hora_fin'], errors='coerce')
resultvalidacion8['Jueves_hora_inicio'] = pd.to_numeric(resultvalidacion8['Jueves_hora_inicio'], errors='coerce')
resultvalidacion8['Viernes_hora_fin'] = pd.to_numeric(resultvalidacion8['Viernes_hora_fin'], errors='coerce')
resultvalidacion8['Viernes_hora_inicio'] = pd.to_numeric(resultvalidacion8['Viernes_hora_inicio'], errors='coerce')
resultvalidacion8['Sabado_hora_fin'] = pd.to_numeric(resultvalidacion8['Sabado_hora_fin'], errors='coerce')
resultvalidacion8['Sabado_hora_inicio'] = pd.to_numeric(resultvalidacion8['Sabado_hora_inicio'], errors='coerce')
resultvalidacion8['Domingo_hora_fin'] = pd.to_numeric(resultvalidacion8['Domingo_hora_fin'], errors='coerce')
resultvalidacion8['Domingo_hora_inicio'] = pd.to_numeric(resultvalidacion8['Domingo_hora_inicio'], errors='coerce')


#resultvalidacion8['Lunes_hora_fin'] = pd.to_datetime(resultvalidacion8['Lunes_hora_fin'], format='%H:%M:%S').dt.time
#resultvalidacion8['Lunes_hora_inicio'] = pd.to_datetime(resultvalidacion8['Lunes_hora_inicio'], errors='coerce').dt.time


resultvalidacion8['dia_semana'] = pd.to_numeric(resultvalidacion8['dia_semana'], errors='coerce')

df_filtrado_validacion8_lunes = resultvalidacion8[(resultvalidacion8['Hora_inicio'] < resultvalidacion8['Lunes_hora_fin'] ) & (resultvalidacion8['Hora_inicio'] >= resultvalidacion8['Lunes_hora_inicio']) & (resultvalidacion8['dia_semana'] == 0)]
falla_lunes=df_filtrado_validacion8_lunes.shape[0]
df_filtrado_validacion8_martes = resultvalidacion8[(resultvalidacion8['Hora_inicio'] < resultvalidacion8['Martes_hora_fin'] ) & (resultvalidacion8['Hora_inicio'] >= resultvalidacion8['Martes_hora_inicio']) & (resultvalidacion8['dia_semana'] == 1)]
falla_martes=df_filtrado_validacion8_martes.shape[0]
df_filtrado_validacion8_miercoles = resultvalidacion8[(resultvalidacion8['Hora_inicio'] < resultvalidacion8['Miercoles_hora_fin'] ) & (resultvalidacion8['Hora_inicio'] >= resultvalidacion8['Miercoles_hora_inicio']) & (resultvalidacion8['dia_semana'] == 2)]
falla_miercoles=df_filtrado_validacion8_miercoles.shape[0]
df_filtrado_validacion8_jueves = resultvalidacion8[(resultvalidacion8['Hora_inicio'] < resultvalidacion8['Jueves_hora_fin'] ) & (resultvalidacion8['Hora_inicio'] >= resultvalidacion8['Jueves_hora_inicio']) & (resultvalidacion8['dia_semana'] == 3)]
falla_jueves=df_filtrado_validacion8_jueves.shape[0]
df_filtrado_validacion8_viernes = resultvalidacion8[(resultvalidacion8['Hora_inicio'] < resultvalidacion8['Viernes_hora_fin'] ) & (resultvalidacion8['Hora_inicio'] >= resultvalidacion8['Viernes_hora_inicio']) & (resultvalidacion8['dia_semana'] == 4)]
falla_viernes=df_filtrado_validacion8_viernes.shape[0]
df_filtrado_validacion8_sabado = resultvalidacion8[(resultvalidacion8['Hora_inicio'] < resultvalidacion8['Sabado_hora_fin'] ) & (resultvalidacion8['Hora_inicio'] >= resultvalidacion8['Sabado_hora_inicio']) & (resultvalidacion8['dia_semana'] == 5)]
falla_sabado=df_filtrado_validacion8_sabado.shape[0]
df_filtrado_validacion8_domingo = resultvalidacion8[(resultvalidacion8['Hora_inicio'] < resultvalidacion8['Domingo_hora_fin'] ) & (resultvalidacion8['Hora_inicio'] >= resultvalidacion8['Domingo_hora_inicio']) & (resultvalidacion8['dia_semana'] == 6)]
falla_domingo=df_filtrado_validacion8_domingo.shape[0]

print (falla_lunes)
print (falla_martes)
print (falla_miercoles)
print (falla_jueves)
print (falla_viernes)
print (falla_sabado)
print (falla_domingo)

#revisa los dias sin horario

df_filtrado_validacion8_lunes_sin_horario = resultvalidacion8[( resultvalidacion8['Lunes_hora_inicio']==0) & (resultvalidacion8['Hora_inicio'] > 0 ) & (resultvalidacion8['fecha_creacion_agenda'] < resultvalidacion8['dateparada']) & (resultvalidacion8['dia_semana'] == 0)]
falla_lunes_sin_horario=df_filtrado_validacion8_lunes_sin_horario.shape[0]
df_filtrado_validacion8_martes_sin_horario = resultvalidacion8[( resultvalidacion8['Martes_hora_inicio']=='00:00:00') & (resultvalidacion8['Hora_inicio'] > 0 ) & (resultvalidacion8['fecha_creacion_agenda'] < resultvalidacion8['dateparada']) & (resultvalidacion8['dia_semana'] == 1)]
falla_martes_sin_horario=df_filtrado_validacion8_martes_sin_horario.shape[0]
df_filtrado_validacion8_miercoles_sin_horario = resultvalidacion8[( resultvalidacion8['Miercoles_hora_inicio']=='00:00:00') & (resultvalidacion8['Hora_inicio'] > 0 ) & (resultvalidacion8['fecha_creacion_agenda'] < resultvalidacion8['dateparada']) & (resultvalidacion8['dia_semana'] == 2)]
falla_miercoles_sin_horario=df_filtrado_validacion8_miercoles_sin_horario.shape[0]
df_filtrado_validacion8_jueves_sin_horario = resultvalidacion8[( resultvalidacion8['Jueves_hora_inicio']=='00:00:00') & (resultvalidacion8['Hora_inicio'] > 0 ) & (resultvalidacion8['fecha_creacion_agenda'] < resultvalidacion8['dateparada']) & (resultvalidacion8['dia_semana'] == 3)]
falla_jueves_sin_horario=df_filtrado_validacion8_jueves_sin_horario.shape[0]
df_filtrado_validacion8_viernes_sin_horario = resultvalidacion8[( resultvalidacion8['Viernes_hora_inicio']=='00:00:00') & (resultvalidacion8['Hora_inicio'] > 0 ) & (resultvalidacion8['fecha_creacion_agenda'] < resultvalidacion8['dateparada']) & (resultvalidacion8['dia_semana'] == 4)]
falla_viernes_sin_horario=df_filtrado_validacion8_viernes_sin_horario.shape[0]
df_filtrado_validacion8_sabado_sin_horario = resultvalidacion8[( resultvalidacion8['Sabado_hora_inicio']=='00:00:00') & (resultvalidacion8['Hora_inicio'] > 0 ) & (resultvalidacion8['fecha_creacion_agenda'] < resultvalidacion8['dateparada']) & (resultvalidacion8['dia_semana'] == 5)]
falla_sabado_sin_horario=df_filtrado_validacion8_sabado_sin_horario.shape[0]
df_filtrado_validacion8_domingo_sin_horario = resultvalidacion8[( resultvalidacion8['Domingo_hora_inicio']=='00:00:00') & (resultvalidacion8['Hora_inicio'] > 0 ) & (resultvalidacion8['fecha_creacion_agenda'] < resultvalidacion8['dateparada']) & (resultvalidacion8['dia_semana'] == 6)]
falla_domingo_sin_horario=df_filtrado_validacion8_domingo_sin_horario.shape[0]

print ("sin horario")
print (falla_lunes_sin_horario)
print (falla_martes_sin_horario)
print (falla_miercoles_sin_horario)
print (falla_jueves_sin_horario)
print (falla_viernes_sin_horario)
print (falla_sabado_sin_horario)
print (falla_domingo_sin_horario)

# Guardar las predicciones en un archivo Excel
#df_filtrado8_paradas_fuera_cd.to_excel('C:/Users/William Wallace/Desktop/dias-horario.xlsx', index=False)
#df_filtrado8_paradas_fuera_cd.to_excel('C:/Users/William Wallace/Desktop/cerrado-fuera-horario2.xlsx', index=False)

fallas_total=falla_lunes+falla_martes+falla_miercoles+falla_jueves+falla_viernes+falla_sabado+falla_domingo
fallas_total_sin_horario=falla_lunes_sin_horario+falla_martes_sin_horario+falla_miercoles_sin_horario+falla_jueves_sin_horario+falla_viernes_sin_horario+falla_sabado_sin_horario+falla_domingo_sin_horario

if fallas_total > 0:
    error8=(f"Total: {fallas_total}")
else:
    error8="OK - Sin Novedades";

if fallas_total_sin_horario > 0:
    print ("Revisar, hay paradas de reloj como si tuviera horario pero no tiene cargado")
    print (fallas_total_sin_horario)
    error8_1="Revisar, hay paradas de reloj como si tuviera horario pero no tiene cargado";
else:
    print ("Las paradas de reloj fuera de horario CD, son correctas")
    error8_1="OK - Sin Novedades";

0
0
0
0
0
0
0
sin horario
0
0
0
0
0
0
0
Las paradas de reloj fuera de horario CD, son correctas


### VALIDACIÓN 9 - CATEGORIA - SUBCATEGORIA - CODIGO CATEGORIA - PRIORIDAD  
Esta validación verifica que la categoria, subcategoria, codigo de categoria y prioridad si corresponda a la suministrada por la lista categorias en el arbol de tipificación  

Retorna:  
*    Cantidad de PQRSD categoria, subcategoria, codigo de categoria y prioridad diferente.  
*    Mensaje de error o exito.

In [ ]:
# Reemplazar NaN con 0
categorias_subcategorias['Column 5'] = categorias_subcategorias['Column 5'].fillna(0)
categorias_subcategorias['Column 5'] = categorias_subcategorias['Column 5'].astype(int)
categorias_subcategorias_concatenado = pd.DataFrame()
categorias_subcategorias_concatenado['Concatenado'] = categorias_subcategorias['Column 1'].astype(str) + categorias_subcategorias['Column 2'].astype(str) + categorias_subcategorias['Column 4'].astype(str) + categorias_subcategorias['Column 5'].astype(str)

# Convertir la columna 'Concatenado' a mayúsculas
categorias_subcategorias_concatenado['Concatenado'] = categorias_subcategorias_concatenado['Concatenado'].str.upper()


id_cate_subca = data[['ID','GRUPO','Estado']].copy()
id_cate_subca['Concatenar'] = data[['CATEGORIA', 'SUBCATEGORIA', 'Código_Categoría', 'PRIORIDAD']].apply(lambda x: ''.join(x.astype(str)), axis=1)

id_cate_subca['Concatenar'] = id_cate_subca['Concatenar'].str.upper()

id_cate_subca['Validación'] = id_cate_subca['Concatenar'].isin(categorias_subcategorias_concatenado['Concatenado'])


error_cate_subca = id_cate_subca[~id_cate_subca['Validación']]

error_cate_subca = error_cate_subca[['ID','GRUPO','Estado']]

if(error_cate_subca.shape[0] > 0):
    error9 = f"Total: <b>{error_cate_subca.shape[0]}</b> <br> {error_cate_subca}"
else:
    error9 = "OK - Sin Novedades"

### VALIDACIÓN 10 Y 11 - FECHA DE CIERRE  
Esta validación verifica que la fecha de cierre del PQRSD no sea mayor ni menor a 1 minuto

Retorna:  
*    Cantidad de PQRSD con error.  
*    Mensaje de error o exito.

In [ ]:
# Validación 10 - Verificar fecha de cierre

# Filtrar el DataFrame por el estado 'Cerrar'
df_filtrado_validacion10_PQRSD_cerrados = dataframe[(dataframe['Estado'] == "Cerrar")]

# Realizar la unión entre los DataFrames utilizando 'ID' y 'uuid' como claves
resultvalidacion10 = pd.merge(df_filtrado_validacion10_PQRSD_cerrados, dataframe_Paradas, left_on='ID', right_on='uuid', how='inner')

# Rellenar los valores NaN con False
resultvalidacion10 = resultvalidacion10.fillna(False)

# Seleccionar las columnas deseadas
columnas_deseadas = ['id_parada', 'uuid', 'idbeneficiario', 'idpqr', 'fecha_ceracion_pqrs', 'Fecha_Cierre_PQRS', 'fecha_cierre_ticket']
resultvalidacion10 = resultvalidacion10[columnas_deseadas]
#print (resultvalidacion10)

# Asegurarse de que las columnas de fecha estén en formato datetime
resultvalidacion10['Fecha_Cierre_PQRS'] = pd.to_datetime(resultvalidacion10['Fecha_Cierre_PQRS'], errors='coerce')
resultvalidacion10['fecha_cierre_ticket'] = pd.to_datetime(resultvalidacion10['fecha_cierre_ticket'], errors='coerce')

# Calcular la diferencia en minutos entre las dos fechas
resultvalidacion10['diferencia_minutos'] = ((resultvalidacion10['Fecha_Cierre_PQRS'] - resultvalidacion10['fecha_cierre_ticket']).dt.total_seconds() / 60)
#print (resultvalidacion10)

# Reemplazar los valores NaN con 0 antes de la conversión a entero

df_filtrado_10_1 = resultvalidacion10[resultvalidacion10['diferencia_minutos'].isnull()]


resultvalidacion10['diferencia_minutos'] = resultvalidacion10['diferencia_minutos'].fillna(0)

# Convertir la diferencia en minutos a entero
resultvalidacion10['diferencia_minutos'] = resultvalidacion10['diferencia_minutos'].astype(int)

# Filtrar las filas donde la diferencia en minutos sea menor que -1 o mayor que 1
df_filtrado_10 = resultvalidacion10[(resultvalidacion10['diferencia_minutos'] < -1) | (resultvalidacion10['diferencia_minutos'] > 1)]

# Contar el número de registros con diferencias
diferencia_cierre = df_filtrado_10.shape[0]

# Contar el número de registros con paradas abiertas y pqrsd cerrado
diferencia_cierre_10_1 = df_filtrado_10_1.shape[0]

#print (diferencia_cierre_10_1)


df_filtrado_10 = df_filtrado_10[['id_parada','uuid','idpqr','diferencia_minutos']]

df_filtrado_10_1 = df_filtrado_10_1[['id_parada','uuid','idpqr','diferencia_minutos']]

# Condicional para imprimir el resultado de la validación
if diferencia_cierre > 0:
    df_filtrado_10 = df_filtrado_10.to_html(index=False)
    error10 = f"Total: {diferencia_cierre} <br> {df_filtrado_10}"
else:
    error10 = "OK - Sin Novedades"

# Condicional para imprimir el resultado de la validación 10.1
if diferencia_cierre_10_1 > 0:
    df_filtrado_10_1 = df_filtrado_10_1.to_html(index=False)
    error10_1 = f"Total: {diferencia_cierre_10_1} <br> {df_filtrado_10_1}"
else:
    error10_1 = "OK - Sin Novedades"



### VALIDACIÓN 12 - PQRS ENVIADAS DATAWIFI VS CREADAS UMBRELLA
Esta validación verifica que la cantdad de PQRS enviadas desde datawifi sea igual a las PQRSD creadas  

Retorna:  
*    Cantidad de PQRSD con error.  
*    Mensaje de error o exito.

In [ ]:
data['Fecha_Creacion'] = pd.to_datetime(data['Fecha_Creacion'])
# Convertir la columna 'Fecha_Creacion' al formato 'YYYY-MM-DD'
data['Fecha_Creacion'] = data['Fecha_Creacion'].dt.strftime('%Y-%m-%d')

log_pqrs_datawifi['FECHA DE CREACIÓN'] = pd.to_datetime(log_pqrs_datawifi['FECHA DE CREACIÓN'])
# Convertir la columna 'Fecha_Creacion' al formato 'YYYY-MM-DD'
log_pqrs_datawifi['FECHA DE CREACIÓN'] = log_pqrs_datawifi['FECHA DE CREACIÓN'].dt.strftime('%Y-%m-%d')

fecha_validacion = fecha.strftime('%Y-%m-%d')

pqrs_datawifi_hoy = log_pqrs_datawifi[(log_pqrs_datawifi['FECHA DE CREACIÓN'] == fecha_validacion)]
pqrs_umbrella_hoy = data[(data['Fecha_Creacion'] == fecha_validacion) & (data['Usuario_Creador'] == 'DATAWIFI.API')]
conteo_pqrs_umbrella_hoy = pqrs_umbrella_hoy.shape[0]
conteo_datawifi = pqrs_datawifi_hoy.shape[0]

if(conteo_pqrs_umbrella_hoy < conteo_datawifi):
    error11 = f"Diferencia: <b>{(conteo_datawifi - conteo_pqrs_umbrella_hoy)}</b> <br> Enviadas DataWifi: <b>{conteo_datawifi}</b> <br> Creadas Umbrella: <b> {conteo_pqrs_umbrella_hoy}</b>"
else:
    error11 = f'OK, Sin Novedades <br> Enviadas DataWifi: <b>{conteo_datawifi}</b> <br> Creadas Umbrella: <b> {conteo_pqrs_umbrella_hoy}</b>'

### VALIDACIÓN 13 - PQRS ENVIADA DATAWIFI SIN UUID
Esta validación verifica que las peticiones enviadas desde datawifi hayan retornado el UUID

Retorna:  
*    Cantidad de PQRSD con error.  
*    Mensaje de error o exito.

In [ ]:
# Filtrar filas donde la columna PQR está vacía
vacios = pqrs_datawifi_hoy[pqrs_datawifi_hoy['PQR'].isna()]

# Crear nuevo DataFrame con las columnas requeridas
vacios = vacios[['IDENTIFICADOR BENEFICIARIO', 'PQR', 'SUBCATEGORYS','STATUS']]
total = vacios.shape[0]

# Verificar si hay filas con errores
if total > 0:
    vacios = vacios.to_html(index=False)
    error12 = (f"Total: <b>{total}</b> <br>{vacios}")
else:
    error12 = "OK - Sin Novedades"

### VALIDACIÓN 14 - ESTADO UMBRELLA vs ESTADO DATAWIFI
Esta validación verifica que el estado del PQRSD en umbrella coincida con el registrado en Datawifi

Retorna:  
*    Cantidad de PQRSD con error.  
*    Mensaje de error o exito.

In [ ]:
#Verificar estados umbrella vs datawifi
log_pqrs_datawifi_estados = log_pqrs_datawifi.copy()
pqrs_estados_umbrella = data.copy()
pqrs_estados_umbrella['Estado'] = pqrs_estados_umbrella['Estado'].replace('Cerrar','Cerrado')
pqrs_estados_umbrella['Estado'] = pqrs_estados_umbrella['Estado'].replace('Agendar','Abierto')
pqrs_estados_umbrella['Estado'] = pqrs_estados_umbrella['Estado'].replace('Asignado Mesa de Ayuda','Abierto')

merged_df = pd.merge(pqrs_estados_umbrella, log_pqrs_datawifi_estados, how='inner', left_on='ID', right_on='PQR')

merged_df = merged_df[merged_df['Estado'] != merged_df['STATUS']]

merged_df = merged_df.rename(columns={'Estado': 'Estado_Umbrella', 'Fecha_Cierre_PQRS':'Fecha_Cierre_PQRS_Umbrella', 'STATUS':'Estado_Datawifi', 'FECHA DE CIERRE':'Fecha_Cierre_PQRS_Datawifi','PQR':'PQRS'})

merged_df = merged_df[['ID_Beneficiario', 'GRUPO','ID', 'Estado_Umbrella', 'Estado_Datawifi','Fecha_Cierre_PQRS_Umbrella', 'Fecha_Cierre_PQRS_Datawifi']]

total = merged_df.shape[0]

if total > 0:
    merged_df = merged_df.to_html(index=False)
    error13 = (f"Total: <b>{total}</b> <br>{merged_df}")
else:
    error13 = "OK - Sin Novedades"

### VALIDACIÓN 15 - ESTADO DATAWIFI vs ESTADO UMBRELLA
Esta validación verifica que el estado del PQRSD en umbrella coincida con el registrado en Datawifi

Retorna:  
*    Cantidad de PQRSD con error.  
*    Mensaje de error o exito.

In [ ]:
#Verificar estados umbrella vs datawifi
log_pqrs_datawifi_estados = log_pqrs_datawifi.copy()
pqrs_estados_umbrella = data.copy()
pqrs_estados_umbrella['Estado'] = pqrs_estados_umbrella['Estado'].replace('Cerrar','Cerrado')
pqrs_estados_umbrella['Estado'] = pqrs_estados_umbrella['Estado'].replace('Agendar','Abierto')
pqrs_estados_umbrella['Estado'] = pqrs_estados_umbrella['Estado'].replace('Asignado Mesa de Ayuda','Abierto')

merged_df = pd.merge(log_pqrs_datawifi_estados, pqrs_estados_umbrella, how='inner', left_on='PQR', right_on='ID')

merged_df = merged_df[merged_df['Estado'] != merged_df['STATUS']]

merged_df = merged_df.rename(columns={'Estado': 'Estado_Umbrella', 'Fecha_Cierre_PQRS':'Fecha_Cierre_PQRS_Umbrella', 'STATUS':'Estado_Datawifi', 'FECHA DE CIERRE':'Fecha_Cierre_PQRS_Datawifi','PQR':'PQRS'})

merged_df = merged_df[['ID_Beneficiario', 'GRUPO','ID', 'Estado_Umbrella', 'Estado_Datawifi','Fecha_Cierre_PQRS_Umbrella', 'Fecha_Cierre_PQRS_Datawifi']]

total = merged_df.shape[0]

if total > 0:
    merged_df = merged_df.to_html(index=False)
    error14 = (f"Total: <b>{total}</b> <br>{merged_df}")
else:
    error14 = "OK - Sin Novedades"

###Validacion 16 Y 17

Esta validación verifica las paradas de reloj Energia Alternativa de PQRSD abiertas y cerradas

Retorna:  
*    Cantidad de PQRSD con error.  
*    Mensaje de error o exito.

In [ ]:
dataframe_horarios_15=dataframe_horarios[['Id_Beneficiario','Tipo_Energia']]

dataframe_15=dataframe[['ID_Beneficiario','Fecha_Creacion','ID','año_fecha_Cierre','mes_fecha_Cierre','dia_fecha_Cierre','Hora_Cierre','Fecha_Cierre_PQRS','Estado']]
dataframe_15 = dataframe_15[dataframe_15['Fecha_Creacion'] > "2024-09-17 00:00:00"]

# Obtener el primer día del mes en curso
primer_dia_mes = datetime.today().replace(day=1)
# Formatear la fecha en el formato 'año-mes-día'
primer_dia_mes = primer_dia_mes.strftime('%Y-%m-%d')
primer_dia_mes_formateado = primer_dia_mes + " 09:09:09"
print (primer_dia_mes)
print (primer_dia_mes_formateado)

puntos_solares = dataframe_horarios_15[(dataframe_horarios_15['Tipo_Energia']=="SOLUCION SOLAR")]
puntos_solares.reset_index(drop=True, inplace=True)
pqrsd_solares=pd.merge(dataframe_15,puntos_solares, left_on='ID_Beneficiario', right_on='Id_Beneficiario', how='inner')

pqrsd_solares.loc[pqrsd_solares['Fecha_Creacion'] < primer_dia_mes, 'Fecha_Creacion'] = primer_dia_mes_formateado
pqrsd_solares['Hora_Creacion'] = pqrsd_solares['Fecha_Creacion'].dt.hour.fillna(-1).astype(int)
pqrsd_solares['dia_Fecha_Creacion'] = pqrsd_solares['Fecha_Creacion'].dt.day.fillna(-1).astype(int)
pqrsd_solares['mes_Fecha_Creacion'] = pqrsd_solares['Fecha_Creacion'].dt.month.fillna(-1).astype(int)
pqrsd_solares['año_Fecha_Creacion'] = pqrsd_solares['Fecha_Creacion'].dt.year.fillna(-1).astype(int)
pqrsd_solares.reset_index(drop=True, inplace=True)

#print (pqrsd_solares)

df_filtrado_validacion15_abiertos = pqrsd_solares[~(pqrsd_solares['Estado'] == "Cerrar")]
df_filtrado_validacion15_cerrados_mes = pqrsd_solares[(pqrsd_solares['Estado'] == "Cerrar") & (pqrsd_solares['año_fecha_Cierre'] == año_actual) & (pqrsd_solares['mes_fecha_Cierre'] == mes_actual)]
paradas_solares = dataframe_Paradas[(dataframe_Paradas['tipo_parada'] == "Energía Alternativa")]

df_filtrado_validacion15_abiertos.reset_index(drop=True, inplace=True)
df_filtrado_validacion15_cerrados_mes.reset_index(drop=True, inplace=True)

#print (df_filtrado_validacion15_abiertos)

#print (df_filtrado_validacion15_cerrados_mes)

cruce_abiertos = pd.merge(df_filtrado_validacion15_abiertos,paradas_solares, left_on='ID', right_on='uuid', how='left')
cruce_cerrados = pd.merge(df_filtrado_validacion15_cerrados_mes,paradas_solares, left_on='ID', right_on='uuid', how='left')
#print (cruce_cerrados)
cruce_abiertos.reset_index(drop=True, inplace=True)
cruce_cerrados.reset_index(drop=True, inplace=True)

#print(cruce_cerrados.columns)
cruce_abiertos['Conteo'] = cruce_abiertos.groupby(['ID', 'ID_Beneficiario', 'Tipo_Energia', 'Estado', 'Fecha_Creacion', 'dia_Fecha_Creacion', 'uuid'])['ID'].transform('count').fillna(0).astype(int)
cruce_cerrados['Conteo'] = cruce_cerrados.groupby(['ID', 'ID_Beneficiario', 'Tipo_Energia',  'Fecha_Creacion', 'dia_Fecha_Creacion','Estado','dia_fecha_Cierre','Fecha_Cierre_PQRS', 'uuid'])['ID'].transform('count').fillna(0).astype(int)

cruce_abiertos=cruce_abiertos[['ID', 'ID_Beneficiario', 'Tipo_Energia', 'Estado', 'Fecha_Creacion', 'dia_Fecha_Creacion','Hora_Creacion', 'uuid','Conteo']]
cruce_cerrados=cruce_cerrados[['ID', 'ID_Beneficiario', 'Tipo_Energia',  'Fecha_Creacion', 'dia_Fecha_Creacion','Hora_Creacion','Estado','dia_fecha_Cierre','Hora_Cierre','Fecha_Cierre_PQRS', 'uuid','Conteo']]

cruce_abiertos.loc[:, 'pardas_calculadas']=(numero_dia-cruce_abiertos['dia_Fecha_Creacion'])
cruce_abiertos.loc[cruce_abiertos['Hora_Creacion'] < 7, 'pardas_calculadas'] = (cruce_abiertos['pardas_calculadas']+1)
cruce_abiertos.loc[:, 'revision']=(cruce_abiertos['pardas_calculadas']-cruce_abiertos['Conteo'])
cruce_cerrados.loc[:, 'pardas_calculadas']=(cruce_cerrados['dia_fecha_Cierre']-cruce_cerrados['dia_Fecha_Creacion'])
cruce_cerrados.loc[cruce_cerrados['Hora_Creacion'] < 7, 'pardas_calculadas'] = (cruce_cerrados['pardas_calculadas']+1)
cruce_cerrados.loc[cruce_cerrados['Hora_Cierre'] > 18, 'pardas_calculadas'] = (cruce_cerrados['pardas_calculadas']+1)

cruce_cerrados['revision']=(cruce_cerrados['pardas_calculadas']-cruce_cerrados['Conteo'])

#print(cruce_abiertos[['Fecha_Creacion',  'Conteo', 'pardas_calculadas', 'revision']])
#print(cruce_cerrados[['Fecha_Creacion',  'Conteo', 'pardas_calculadas', 'revision','Hora_Cierre']])

fallas_abiertos = cruce_abiertos[cruce_abiertos['revision'] !=0 ]
fallas_cerrados = cruce_cerrados[cruce_cerrados['revision'] !=0 ]

fallas_abiertos = fallas_abiertos.groupby(['ID', 'ID_Beneficiario', 'Tipo_Energia', 'Estado', 'Fecha_Creacion', 'dia_Fecha_Creacion', 'uuid']).last().reset_index()
fallas_cerrados = fallas_cerrados.groupby(['ID', 'ID_Beneficiario', 'Tipo_Energia', 'Estado', 'Fecha_Creacion', 'dia_Fecha_Creacion', 'uuid']).last().reset_index()



total_fallas_abiertos = fallas_abiertos.shape[0]
total_fallas_cerrados = fallas_cerrados.shape[0]

if total_fallas_abiertos > 0:
    fallas_abiertos = fallas_abiertos.to_html(index=False)
    error15_abiertos=(f"Total: <b>{total_fallas_abiertos}</b> <br>{fallas_abiertos}")
else:
    error15_abiertos = "OK - Sin Novedades"

if total_fallas_cerrados > 0:
    fallas_cerrados = fallas_cerrados.to_html(index=False)
    error15_cerrados = (f"Total: <b>{total_fallas_cerrados}</b> <br>{fallas_cerrados}")
else:
    error15_cerrados = "OK - Sin Novedades"

#print (total_fallas_abiertos)
#print (fallas_cerrados)




2024-12-01
2024-12-01 09:09:09


## CONSOLIDADO ERRORES


In [ ]:
print ("validacion 1 PQRSD SIN JOBID")
print (error1)
print ("validacion 2 PQRSD SiN FECHA MAXIMA")
print (error2)
print ("validacion 3 PQRSD AGENDADO SIN CORRECTIVO")
print (error3)
print ("validacion 4 Fecha de cierre - verificar cierres fuera de horario")
print (error4)
print ("validacion 5 pqrsd agendados sin parada de reloj fuera de horario cd")
print (error5)
print ("validacion 6 pqrsd en MDA sin parada de reloj fuera de horario MDA")
print (error6)
print ("validacion 7 tiempo de paradas de reloj")
print (error7)
print ("validacion 8 paradas de fuera de horario creadas dentro del horario cd")
print (error8)
print ("validacion 8.1 paradas creadas como si tuviera horario pero ese dia no tiene horario cargado")
print (error8_1)
print ("validacion 9 categoria, subcategoria, codigo de categoria y prioridad")
print (error9)
print ("validacion 10 verificar fecha cierre")
print (error10)
print ("validacion 11 verificar fecha cierre de las paradas de reLoj")
print (error10_1)
print ("validacion 12 Enviadas DataWifi vs Creadas en Umbrella")
print (error11)
print ("validacion 13 Enviadas DataWifi sin UUID")
print (error12)
print ("validacion 14 Estado Umbrella vs Estado Datawifi")
print (error13)
print ("validacion 15 Estado Datawifi vs Estado Umbrella")
print (error14)
print ("validacion 16 Parada Energía Alternativa - PQRSD Abiertos")
print (error15_abiertos)
print ("validacion 17 Parada Energía Alternativa - PQRSD Cerrados")
print (error15_cerrados)


validacion 1 PQRSD SIN JOBID
OK - Sin Novedades
validacion 2 PQRSD SiN FECHA MAXIMA
OK - Sin Novedades
validacion 3 PQRSD AGENDADO SIN CORRECTIVO
OK - Sin Novedades
validacion 4 Fecha de cierre - verificar cierres fuera de horario
Total: <b>4</b> <br><table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>ID</th>
      <th>GRUPO</th>
      <th>PRIORIDAD</th>
      <th>Fecha_Creacion</th>
      <th>Fecha_Cierre_PQRS</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>66316E-3345BC</td>
      <td>GRUPO 3</td>
      <td>2</td>
      <td>2024-12-08 07:04:02</td>
      <td>2024-12-09 23:01:16</td>
    </tr>
    <tr>
      <td>253EFD-DC7896</td>
      <td>GRUPO 2</td>
      <td>2</td>
      <td>2024-12-07 07:03:00</td>
      <td>2024-12-08 21:01:54</td>
    </tr>
    <tr>
      <td>4F4592-042CFB</td>
      <td>SALIENTE GRUPO 2</td>
      <td>1</td>
      <td>2024-10-29 07:07:21</td>
      <td>2024-12-03 23:59:00</td>
    </tr>
    <tr>
      <td>64147F

## ENVIO CORREO

In [ ]:
nombres = ['Victor Campos', 'Liseth Guzmán', 'William Mora']

while True:
    for i, nombre in enumerate(nombres, 1):
        print(f"{i}. {nombre}")

    opcion = int(input("Seleccione una opcion: "))

    if 1 <= opcion <= len(nombres):
        nombre = nombres[opcion - 1]
        print(f"Has seleccionado: {nombre}")
        break  # Salir del bucle cuando se selecciona una opción válida
    else:
        print("Opción no válida, por favor intente de nuevo.\n")

# Variable de fecha

hora = datetime.now(pytz.timezone('America/Bogota')).strftime("%H:%M:%S")

# Configuración del servidor SMTP
email_subject = f"Revisiones diarias {fecha}"
sender_email_address = "wsistemasdeinformacion@gmail.com"
receiver_email_address = "sistemasdeinformacionUT@sky.net.co, nelson.miranda@sky.net.co"
email_smtp = "smtp.gmail.com"
email_password = "vkkb rlcg hhfp ppuo"

# Creación del objeto EmailMessage
message = EmailMessage()

# Configuracion de las cabeceras del email
message['Subject'] = email_subject
message['From'] = sender_email_address
message['To'] = receiver_email_address

# Creacion del cuerpo del correo
body = f"""
<html>
<head>
    <style>
        body {{
            font-family: Arial, sans-serif;
            font-size: 16px;
            color: #333333;
        }}
        h1 {{
            color: #000000;
            font-size: 24px;
        }}
        h3 {{
            color: #333333;
            font-size: 20px;
        }}
        p {{
            font-size: 16px;
            color: #555555;
        }}
        .highlight {{
            color: #FF5733;
            font-weight: bold;
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
        }}
        th {{
            background-color: #003366;
            color: white;
            text-align: center;
            padding: 8px;
        }}
        td {{
            text-align: center;
            padding: 8px;
            border: 1px solid #dddddd;
        }}
        tr:nth-child(even) {{
            background-color: #f2f2f2;
        }}
    </style>
</head>
<body>
    <h1>VALIDACIONES DIARIAS {fecha}</h1>
    <h3 class="highlight">Validación 1</h3>
    <p><b>PQRSD SIN JOBID: </b>{error1}</p>
    <h3 class="highlight">Validación 2</h3>
    <p><b>PQRSD SIN FECHA MAXIMA DE ATENCIÓN: </b>{error2}</p>
    <h3 class="highlight">Validación 3</h3>
    <p><b>PQRSD AGENDADO SIN CORRECTIVO: </b>{error3}</p>
    <h3 class="highlight">Validación 4</h3>
    <p><b>CIERRES DE PQRSD FUERA DE HORARIO: </b>{error4}</p>
    <h3 class="highlight">Validación 5</h3>
    <p><b>PQRSD AGENDADOS SIN PARADA DE RELOJ FUERA DE HORARIO CD: </b>{error5}</p>
    <h3 class="highlight">Validación 6</h3>
    <p><b>PQRSD ESTADO ASIGNADO A MDA SIN PARADA DE RELOJ FUERA HORARIO MDA: </b>{error6}</p>
    <h3 class="highlight">Validación 7</h3>
    <p><b>PARADAS DE RELOJ MAYOR A 24H: </b>{error7}</p>
    <h3 class="highlight">Validación 8</h3>
    <p><b>PARADAS DE RELOJ FUERA DE HORARIO CD - CUANDO EL CD NO TIENE HORARIOS: </b>{error8}</p>
    <h3 class="highlight">Validación 9</h3>
    <p><b>CATEGORIA - SUBCATEGORIA - CODIGO CATEGORIA - PRIORIDAD: </b>{error9}</p>
    <h3 class="highlight">Validación 10</h3>
    <p><b>FECHA DE CIERRE PARADA RELOJ VS PQRSD DIFERENCIA 2 MINUTOS: </b>{error10}</p>
     <h3 class="highlight">Validación 11</h3>
    <p><b>FECHA DE CIERRE PARADA RELOJ VS PQRSD ESTADOS DIFERENTES: </b>{error10_1}</p>
    <h3 class="highlight">Validación 12</h3>
    <p><b>ENVIADAS DATAWIFI VS CREADAS EN UMBRELLA: </b>{error11}</p>
    <h3 class="highlight">Validación 13</h3>
    <p><b>ENVIADAS DATAWIFI SIN UUID: </b>{error12}</p>
    <h3 class="highlight">Validación 14</h3>
    <p><b>ESTADO UMBRELLA vs ESTADO DATAWIFI: </b>{error13}</p>
    <h3 class="highlight">Validación 15</h3>
    <p><b>ESTADO DATAWIFI vs ESTADO UMBRELLA: </b>{error14}</p>
    <h3 class="highlight">Validación 16</h3>
    <p><b>PARADA ENERGÍA ALTERNATIVA - PQRSD ABIERTOS: </b>{error15_abiertos}</p>
    <h3 class="highlight">Validación 17</h3>
    <p><b>PARADA ENERGÍA ALTERNATIVA - PQRSD CERRADOS: </b>{error15_cerrados}</p>
    <br>

    <p>Proceso automatizado ejecutado por: <b>{nombre}</b> a las {hora}</p>
</body>
</html>
"""

# Agregar el contenido HTML al mensaje
message.add_alternative(body, subtype='html')

# Variable server guarda configuracion del protocolo y puerto
server = smtplib.SMTP(email_smtp, '587')

# Identificación del cliente en el servidor SMTP
server.ehlo()

# Añade seguridad a la conexión
server.starttls()

# Inicio de sesion con las credenciales
server.login(sender_email_address, email_password)

# Envio correo
server.send_message(message)

# Cerrar conexión con el servidor
server.quit()

1. Victor Campos
2. Liseth Guzmán
3. William Mora
Seleccione una opcion: 1
Has seleccionado: Victor Campos


(221,
 b'2.0.0 closing connection 8926c6da1cb9f-4e2ca310cfbsm2171508173.152 - gsmtp')